#### Brief Descriptions of Attributes in Dataset

    Unique_ID : Unique Identifier.
    Name : Name of the Artist.
    Genre : Genre of the Song.
    Country : Origin Country of Artist.
    Song_Name : Name of the Song.
    Timestamp : Release Date and Time.
    Views : Number of times the song was played/viewed (*Target/Dependent Variable*).
    Comments : Count of comments for the song.
    Likes : Count of Likes.
    Popularity : Popularity score for the artist.
    Followers : Number of Followers.

#### Files description:

    Data_Train.csv – the training set, 78458 rows with 11 columns.
    Data_Test.csv – the test set, 19615 rows with 10 columns, except the Views column.
   
<h5>I am going to explain all my approaches whether it gives best result or not.</h5>
##### Approach 1:
<ul>
<li>I just convert non-numeric features in float.
<li>Extract year , month, day, week_day from timestamp features.
<li>On the basis of corelation matrix, I build my first model.
<li>As most classical algorithm for regression problem, I applied LinearReggression which provides
good result to start with decent Rank.
</ul>

##### Approach 2 :
<ul>
<li>Now, after some EDA I got an idea about the relation between other features with target.
<li>Did one-hot encoding on 'Genre'.
<li>Simply applied RandomForest.
<li>Extracted only important features though feature_importances_ in RandomForest which contains
all numeric features, year,month, couple of Genres.
</ul>

##### Approach 3 :
<ul>
<li>Then, I used stacked ensemble model where my stack of algorithms is - RandomForestRegressor, CatBoostRegressor<br>LGBMRegressor and XGBoostRegressor.         
<li>After performing hyperparameter tuning with the help of GridSearch , this model helps me to reduce my RMSE too much.
</ul>

##### Approach 4 :
<ul>
<li>We know that as much as data we have our model is going to learn better and I tried some data augmentation techniques too.
</ul>
Then I observe something interesting in this dataset :
<img src="Blank_Diagram.png"  height="400" width="550"/>
Iterating over this loop is going to converge towards lower RMSE. This doesn't mean that if we perform more <br>
iterations then our RMSE becomes zero. At some point things is going worse due to this looping and needs some <br>other features and hyperparameter tuning.
<br>This time I just used XGBoostRegressor.

### Imports

In [ ]:
import pickle
import numpy as np
import regex as re
import pandas as pd
import seaborn as sb
import datetime as dt
import xgboost as xgb

from math import sqrt
from datetime import date
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Mounting google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Introducing Train_set

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Data_Train.csv')

In [ ]:
df.drop(columns=['Country'],inplace=True)

In [ ]:
# Convert Timestamp column into datetime datatype
df['Timestamp'] = pd.to_datetime(df['Timestamp']).dt.strftime('%Y-%m-%d %H:%I:%S')
#set Timestamp as index
df.index = df['Timestamp']
#drop Timestamp column from original dataset
#df.drop(columns=['Timestamp'],inplace=True)

#sorting by index
df.sort_index(inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.describe().T

In [ ]:
plt.figure(figsize=(20,4))
plt.hist(df['Genre'],bins=21)

In [ ]:
df.groupby('Name').mean()['Views'].sort_values().plot(kind='bar',figsize=(10,4))
plt.title("Distribution of Views, Name-wise")
#plt.savefig('/content/drive/My Drive/Distribution-of-Views-Name-wise.jpg')

In [ ]:
df.groupby('Genre').count()['Views'].sort_values()[-10:]

In [ ]:
tmp = pd.pivot_table(df,values='Comments',index='Genre',columns=['Year'], aggfunc=np.mean, fill_value=0)
tmp.head()

In [ ]:
plt.figure(figsize=(20,25))
i=1
for col in tmp.columns:
    plt.subplot(7,3,i)
    tmp[col].plot()
    plt.title(col,fontsize=14)
    plt.xlabel("")
    i+=1

plt.savefig('/content/drive/My Drive/Genre-Year-Wise-Distibution.jpg')

## Feature Engineering :

In [ ]:
df['Year'] = date.today().year - pd.DatetimeIndex(df.index).year  #New Feature - Year Number
df['Month'] = pd.DatetimeIndex(df.index).month                    #New Feature - Month Number
df['day_name'] = pd.DatetimeIndex(df.index).dayofweek             #New Feature - Day of Week Number

In [ ]:
# Log
df['Log_likes'] = np.log(df['Likes'])
df['Log_Comments'] = np.log(df['Comments'])
df['Log_Followers'] = np.log(df['Followers'])
df['Log_Popularity'] = np.log(df['Popularity'])

# SQRT
df['Sqrt_likes'] = np.sqrt(df['Likes'])
df['Sqrt_Comments'] = np.sqrt(df['Comments'])
df['Sqrt_Followers'] = np.sqrt(df['Followers'])
df['Sqrt_Popularity'] = np.sqrt(df['Popularity'])

# Standarize
df['Std_likes'] = (df['Likes']  - df['Likes'].mean())/df['Likes'].std()
df['Std_Comments'] = (df['Comments']-df['Comments'].mean())/df['Comments'].std()
df['Std_Followers'] = (df['Followers'] -df['Followers'].mean() )/df['Followers'].std()
df['Std_Popularity'] = (df['Popularity'] -df['Popularity'].mean() )/df['Popularity'].std()


In [ ]:
#Year-wise Genre
tmp = pd.pivot_table(df,values='Comments',index='Genre',columns=['Year'], aggfunc='count', fill_value=0)
tmp.head()
for index in tmp.index:
  for col in tmp.columns:
    df.loc[(df['Genre']==index)  & (df['Year']==col),'Genre_year'] = list(tmp[tmp.index==index][col])*int(tmp[tmp.index==index][col])

#Name-wise Genre
tmp = pd.pivot_table(df,values='Comments',index='Name',columns=['Year'], aggfunc='count', fill_value=0)
tmp.head()
for index in tmp.index:
  for col in tmp.columns:
    df.loc[(df['Name']==index)  & (df['Year']==col),'Name_year'] = list(tmp[tmp.index==index][col])*int(tmp[tmp.index==index][col])

#Month-wise Genre
tmp = pd.pivot_table(df,values='Comments',index='Genre',columns=['Month'], aggfunc='count', fill_value=0)
tmp.head()
for index in tmp.index:
  for col in tmp.columns:
    df.loc[(df['Genre']==index)  & (df['Month']==col),'Genre_month'] = list(tmp[tmp.index==index][col])*int(tmp[tmp.index==index][col])

#Month-wise Name
tmp = pd.pivot_table(df,values='Comments',index='Name',columns=['Month'], aggfunc='count', fill_value=0)
tmp.head()
for index in tmp.index:
  for col in tmp.columns:
    df.loc[(df['Name']==index)  & (df['Month']==col),'Name_month'] = list(tmp[tmp.index==index][col])*int(tmp[tmp.index==index][col])

df = df.replace([np.inf,-np.inf],np.nan)

In [ ]:
df.isna().sum()

In [ ]:
df = df.fillna(0)

## Preprocessing


In [ ]:
def makeNumeric(data,col):
    df[col] = df[col].str.replace(',','')
    likes = []
    for d in df[col]:
        if d.endswith('K'):
            likes.append(float(d.replace('K',''))*1000)
        elif d.endswith('M'):
            likes.append(float(d.replace('M',''))*1000000)
        else:
            likes.append(float(d))
    return likes

In [ ]:
df['Likes'] = makeNumeric(df,'Likes')

In [ ]:
df['Popularity'] = makeNumeric(df,'Popularity')

### One-hot Encoding of Genre

In [ ]:
df = pd.concat([df,pd.get_dummies(df['Genre'])] ,axis=1 )

In [ ]:
df.drop(columns=['Genre'],inplace=True)

### One-hot encoding Name

In [ ]:
df['Name'] = df['Name'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)  
# removing emojis and special characters

In [ ]:
df = pd.concat([df,pd.get_dummies(df['Name'])] ,axis=1 )

In [ ]:
#extracting top words occuring in Name feature
df['Name'] = df['Name'].str.lower()
top_name  =  pd.Series(' '.join(df['Name']).lower().split()).value_counts()[:100]
top_name = top_name.index

for top in reversed(top_name):
    df.loc[(df['Name'].str.contains(str(top))==True),'Top_name'] = top


df = pd.concat([df,pd.get_dummies(df['Top_name'])] ,axis=1 )

### Coorelation Heatmap :

In [ ]:
plt.figure(figsize=(20,15))
sb.heatmap(df.corr(),annot=True,cmap='Blues')

### Relation among all features :

In [ ]:
sb.pairplot(df[['Views',
       'Comments', 'Likes', 'Popularity', 'Followers', 'Year', 'Month',
       'day_name']],diag_kind='kde')

### Distribution of numeric features over the timestamp

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(3,2,1)
df['Views'].plot()
plt.title('Views')
plt.xlabel("")

plt.subplot(3,2,2)
df['Comments'].plot()
plt.title('Comments')
plt.xlabel("")

plt.subplot(3,2,3)
df['Popularity'].plot()
plt.title('Popularity')
plt.xlabel("")

plt.subplot(3,2,4)
df['Likes'].plot()
plt.title('Likes')
plt.xlabel("")

plt.subplot(3,2,5)
df['Followers'].plot()
plt.title('Followers')
plt.xlabel("")



### Other Features Vs Views

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter(df['Likes'],df['Views'])

In [ ]:
df = df[df['Likes']<=2500000] #removing Outlier

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter((df['Comments']),df['Views'])

In [ ]:
df = df[df['Comments']<=80000] #removing Outlier

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter((df['Popularity']),df['Views'])

In [ ]:
df = df[df['Popularity']<=250000] #removing Outlier

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter((df['Followers']),df['Views'])

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter(df['Month'],df['Views'])

In [ ]:
plt.figure(figsize=(16,5))
plt.scatter(df['Year'],df['Views'])

### Box Plots

In [ ]:
df['Likes'].plot(kind='box')

In [ ]:
df['Popularity'].plot(kind='box')

In [ ]:
df['Comments'].plot('box')

In [ ]:
df['Followers'].plot(kind='box')

## Splitting dataset

In [ ]:
df.columns

In [ ]:
X = df[['Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']]
Y = df['Views']

In [ ]:
xtrain, xtest,ytrain,ytest = train_test_split(X,Y)

## Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# I used 3 features at a time but here, I wrote it in once.
subsample = [1,0.8,0.7]
gamma = [0,3,5]
colsample_bytree = [1,0.8,0.7]
learning rate = [0.1,0.05,0.03]
n_estimator = [100,500,1000]
max_depth = [5,6,7,8]
param_grid = dict(subsample =subsample,
                  gamma=gamma,
                  colsample_bytree=colsample_bytree
                  learning_rate = learning_rate,
                  n_estimator = n_estimator,
                  max_depth = max_depth)

In [ ]:
grid_search = GridSearchCV(model, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, cv=5,verbose=12)

In [ ]:
grid_result = grid_search.fit(X, Y)

In [ ]:
grid_result.best_params_

## Model Training

In [ ]:
model1= xgb.XGBRegressor(learning_rate=0.1,
                        n_estimators=500,
                        max_depth=8,
                        colsample_bytree=0.8,
                        subsample=0.7
                        booster='gbtree',
                        objective='reg:squarederror',
                        )

In [ ]:
model1.fit(xtrain,ytrain, eval_set=[(xtest,ytest)])

### Getting Importance Features

In [ ]:
important_featues = pd.DataFrame(model1.get_booster().get_score().items(), columns=['feature','importance']).sort_values('importance', ascending=False)

In [ ]:
important_featues['feature'][:30].values #top 30 features

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(model1,max_num_features=30,ax=ax)

## Introducing Test_Set

In [ ]:
testSet = pd.read_csv('/content/drive/My Drive/Data_Test.csv')

### Preprocessing

In [ ]:
# Convert Timestamp column into datetime datatype
testSet['Timestamp'] = pd.to_datetime(testSet['Timestamp']).dt.strftime('%Y-%m-%d %H:%I:%S')
#set Timestamp as index
testSet.index = testSet['Timestamp']
#drop Timestamp column from original dataset
testSet.drop(columns=['Timestamp'],inplace=True)

#sorting by index
testSet.sort_index(inplace=True)

In [ ]:
testSet['Likes'] = makeNumeric(testSet,'Likes')
testSet['Popularity'] = makeNumeric(testSet,'Popularity')

### Features Creation

In [ ]:
testSet.describe().T

In [ ]:
testSet['Year'] = date.today().year- pd.DatetimeIndex(testSet.index).year

In [ ]:
testSet['Month'] = pd.DatetimeIndex(testSet.index).month

In [ ]:
testSet['day_name'] = pd.DatetimeIndex(testSet.index).dayofweek

In [ ]:
testSet = pd.concat([testSet,pd.get_dummies(testSet['Genre'])] ,axis=1 )

In [ ]:
testSet['Name'] = testSet['Name'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)
testSet['Name'] = testSet['Name'].str.lower()
top_name  =  pd.Series(' '.join(testSet['Name']).lower().split()).value_counts()[:100]
top_name = top_name.index

for top in reversed(top_name):
    testSet.loc[(testSet['Name'].str.contains(str(top))==True),'Top_name'] = top


testSet = pd.concat([testSet,pd.get_dummies(testSet['Top_name'])] ,axis=1 )

In [ ]:
testSet['Name'] = testSet['Name'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

In [ ]:
testSet = pd.concat([testSet,pd.get_dummies(testSet['Name'])] ,axis=1 )

### Comparison of Distribution of Trainset and Testset features:

In [ ]:
plt.figure(figsize=(22,10))

plt.subplot(2,4,1)
testSet['Comments'].plot(color='red')
plt.title('Comments')
plt.xticks([], [])

plt.subplot(2,4,2)
testSet['Popularity'].plot(color='orange')
plt.title('Popularity')
plt.xticks([], [])

plt.subplot(2,4,3)
testSet['Likes'].plot(color='purple')
plt.title('Likes')
plt.xticks([], [])

plt.subplot(2,4,4)
testSet['Followers'].plot(color='green')
plt.title('Followers')
plt.xticks([], [])


plt.subplot(2,4,5)
df['Comments'].plot(color='red')
plt.title('Comments')
plt.xticks([], [])

plt.subplot(2,4,6)
df['Popularity'].plot(color='orange')
plt.title('Popularity')
plt.xticks([], [])

plt.subplot(2,4,7)
df['Likes'].plot(color='purple')
plt.title('Likes')
plt.xticks([], [])

plt.subplot(2,4,8)
df['Followers'].plot(color='green')
plt.title('Followers')
plt.xticks([], [])


### Predicting Views

In [ ]:
testInput = testSet[[ 'Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']]

In [ ]:
testSet['Views'] = model1.predict(testInput)  

### Comparision of Views Vs other Features of both train and test set

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.scatter(testSet['Popularity'],testSet['Views'])

plt.subplot(1,2,2)
plt.scatter(df['Popularity'],df['Views'])

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.scatter(testSet['Likes'],testSet['Views'])

plt.subplot(1,2,2)
plt.scatter(df['Likes'],df['Views'])

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.scatter(testSet['Followers'],testSet['Views'])

plt.subplot(1,2,2)
plt.scatter(df['Followers'],df['Views'])

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.scatter(testSet['Comments'],testSet['Views'])

plt.subplot(1,2,2)
plt.scatter(df['Comments'],df['Views'])

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
plt.scatter(testSet['Unique_ID'],testSet['Views'])

plt.subplot(1,2,2)
plt.scatter(df['Unique_ID'],df['Views'])

### Creating Dataframe for Submission

In [ ]:
predtest = pd.DataFrame({'Unique_ID':testSet['Unique_ID'],
                     'Views': testSet['Views'] })

In [ ]:
predtest['Views'] = predtest['Views'].astype(int)

### Distribution of Views Train Vs Test set

In [ ]:
plt.figure(figsize=(16,4))
plt.subplot(1,2,1)
predtest['Views'].plot()
plt.xticks([],[])

plt.subplot(1,2,2)
df['Views'].plot()
plt.xticks([],[])

In [ ]:
predtest.to_excel('/content/drive/My Drive/Submission1.xlsx',index=False) # Excel file to be submitted

In [ ]:
tmp = pd.read_excel('/content/drive/My Drive/Submission1.xlsx')

In [ ]:
testSet.loc[:,'Views'] =  np.array(tmp['Views'])

### Implementation Approach 4 :

In [ ]:
views = np.array(tmp['Views'])
rmse = []
for i in range(5):
    print('\n\n') 
    print(i+1)
    testSet.loc[:,'Views'] =  views
    frameds = [df[[ 'Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']],
           testSet[[ 'Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']]]
    #concatenation of trainset and (testset+predicted_target)
    final = pd.concat(frameds)
    
    X = final[[ 'Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']]
    Y = final['Views']

    model1.fit(X,Y, eval_set=[(xtest,ytest)])
  
    testInput = testSet[[ 'Unique_ID', 'Likes', 'Comments', 'Popularity', 'Followers',
       'Year', 'all-music','lil', 'electronic', 'music','danceedm', 'rbsoul', 'latin', 'pop', 'dha',
       'rock', 'dj', 'do', 'classical', 'r3hab', 'mad','steve', 'monstercat']]
    testSet['Views'] = model1.predict(testInput)  
    
    # these two are manually derive some outlier that should be in prediction
    testSet.loc[(testSet['Popularity']>100000) & (testSet['Views']>170000000),'Views'] = 195000000
    testSet.loc[testSet['Unique_ID']==86556,'Views'] = 135233520
    views = testSet['Views'].astype(int)

In [ ]:
predtest = pd.DataFrame({'Unique_ID':testSet['Unique_ID'],
                     'Views': views })

In [ ]:
predtest.describe().T

In [ ]:
predtest.to_excel('/content/drive/My Drive/Submission2.xlsx',index=False) # Excel file to be submitted